In [2]:
#MODULOS DE SELENIUM
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime

#MODULOS SECUNDARIOS
import pandas as pd
import numpy as np
import os
import time
import ssl
import sys
import traceback
import pyautogui


#MODULO ALERTAS DESACTIVADAS
import warnings
warnings.filterwarnings('ignore')

#MODULO PROPIOS
from utils import modelamiento

In [2]:
# try:
#     # Obtener tamaño del monitor
#     ancho, alto = pyautogui.size()

#     while True:
#         # Posición actual del cursor
#         x, y = pyautogui.position()

#         # Cálculo proporcional
#         porcentaje_x = x / ancho
#         porcentaje_y = y / alto

#         print(
#             f"Posición: X={x}, Y={y} | "
#             f"Relativo: X={porcentaje_x:.2%}, Y={porcentaje_y:.2%}",
#             end="\r"
#         )

#         time.sleep(0.1)

# except KeyboardInterrupt:
#     print("\nDetenido por el usuario.")

In [3]:
#CAPTURA DE LA LISTA DE POSTS
# URL_POST  = pd.read_excel('url/URL_POST.xlsx') 
# lista_url = list(URL_POST['url_facebook'])

In [4]:
# print("moviendo")
# pyautogui.moveTo(x=785, y=879, duration=0.5)
# pyautogui.click()

# for _ in range(3):
#     pyautogui.scroll(-300)
#     time.sleep(2.5)

In [5]:
#FUNCION POST / VIDEO FACEBOOK

def ManagerFaceBookScrapper (url,medio):

    driver = webdriver.Firefox()
    driver.get(url)

# Publicacion Multimedia - Video (No Reel)
    try:
        
        time.sleep(2)
        # Espera a que el botón sea clickeable
        boton = driver.find_element(By.XPATH, value="/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/div").click()
  
    # Detener el video    
        ## Coordenadas
        x = 83
        y = 256
        ## Duración del movimiento en segundos
        duracion = 0.5
        pyautogui.moveTo(x, y, duration=duracion)
        pyautogui.click()
        
        time.sleep(1)
        print('video detenido')

    # Todos los comentarios
        try:
            # Seleccion de "Todos los comentarios"
            time.sleep(1)
            # Espera a que el botón sea clickeable
            boton = driver.find_element(By.XPATH, "//div[@role='button' and contains(., 'Más pertinentes')]")
            boton.click()

            time.sleep(2)
            boton = driver.find_element(By.XPATH, "//span[contains(text(), 'Todos los comentarios')]")
            boton.click()

            # Comenzar scroll de contenedor de comentarios y click boton "Ver Mas"
            time.sleep(2)
            boton = driver.find_element(By.XPATH, "//span[contains(text(), 'Ver más comentarios')]")
            boton.click()
            print('click ver mas')

            pyautogui.moveTo(x=1261, y=374, duration=0.5)
            pyautogui.click()

            for _ in range(3):
                pyautogui.scroll(-300)
                time.sleep(2.5)

##
            intentos = 0
            max_intentos = 8
            intentos_sin_boton = 0
            seguir = True  # variable de control para el bucle

            while seguir and intentos < max_intentos:
                try:
                    boton = driver.find_element(By.XPATH, "//span[contains(text(), 'Ver más comentarios')]")
                    boton.click()
                    time.sleep(1)
                    intentos_sin_boton = 0  # reinicia el contador si encuentra el botón
                except:
                    for _ in range(3):
                        pyautogui.scroll(-500)
                        time.sleep(1)
                    intentos += 1
                    intentos_sin_boton += 1

                    # Si no encuentra el botón 1 veces seguidas, termina el bucle cambiando la variable
                    if intentos_sin_boton >= 1:
                        print("No se encontró el botón en 1 intentos consecutivos. Saliendo del bucle.")
                        seguir = False

            if intentos == max_intentos:
                print("No se encontró el botón después de 8 intentos.")

##
        except Exception as e:
            pass





        # for _ in range(80):
        #     pyautogui.scroll(-300)
        #     time.sleep(2.5)



# Contenedor de Comentarios

        comentarios = driver.find_elements(By.CSS_SELECTOR,"div.xv55zj0.x1vvkbs.x1rg5ohu.xxymvpz") #CLASE DEL CONTENEDOR DE NOMBRE DE USUARIO, COMENTARIO, HORA Y REACCIONES
        print(comentarios)

### div.xv55zj0.x1vvkbs.x1rg5ohu.xxymvpz CLASE EN LOS VIDEOS


# Publicacion sin Multimedia (Post/Imagen)
    except:
        try:
            # Seleccion de "Todos los comentarios"
            time.sleep(1)
            # Espera a que el botón sea clickeable
            boton = driver.find_element(By.XPATH, "//div[@role='button' and contains(., 'Más pertinentes')]")
            boton.click()

            time.sleep(2)
            boton = driver.find_element(By.XPATH, "//span[contains(text(), 'Todos los comentarios')]")
            boton.click()

        except Exception as e:
            pass
            # print("no_button")
            # print("Ocurrió un error:", e)

        time.sleep(10)

        # Encontrar todos los elementos con la clase específica
        classCSScomentarios_ = "div.x1r8uery.x1iyjqo2.x6ikm8r.x10wlt62.x1pi30zi"

        comentarios = driver.find_elements(By.CSS_SELECTOR,classCSScomentarios_) #CLASE DEL CONTENEDOR DE NOMBRE DE USUARIO, COMENTARIO, HORA Y REACCIONES


## Extraer los textos de los comentarios
 
    textos_comentarios = [comentario.text for comentario in comentarios]

    try:
        print("Iniciando procesamiento de textos_comentarios...")
        procesados = [modelamiento(texto) for texto in textos_comentarios]
        print(f"Procesados: {procesados}")

        flattened_data = [item for sublist in procesados for item in sublist]
        print(f"Datos aplanados: {flattened_data}")

        print("Creando DataFrame...")
        df = pd.DataFrame(flattened_data, columns=["Usuario", "Comentario", "Hora", "Reacciones"])
        print(f"DataFrame creado con {len(df)} filas")

        df["url_facebook"] = url
        print(f"Columna 'url_facebook' añadida con valor: {url}")

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'data/data_{medio}_{timestamp}.xlsx'
        print(f"Guardando archivo Excel en: {filename}")

        df.to_excel(filename, index=False)
        print("Archivo guardado correctamente.")

    except Exception as e:
        print(f"Error detectado: {e}")



    # driver.quit()
    print("Excel generado con éxito 🎉")   
    

In [6]:
url = 'https://www.facebook.com/adncl/posts/pfbid0dimR8jJYrbtkjZ7Vq2kufnMpa5r1Xty38mohvTWT7ssvJnQVDtgYDhTQ8EDiy3KZl'

comentarios = ManagerFaceBookScrapper(url,medio="medios")

Iniciando procesamiento de textos_comentarios...
Procesados: []
Datos aplanados: []
Creando DataFrame...
DataFrame creado con 0 filas
Columna 'url_facebook' añadida con valor: https://www.facebook.com/adncl/posts/pfbid0dimR8jJYrbtkjZ7Vq2kufnMpa5r1Xty38mohvTWT7ssvJnQVDtgYDhTQ8EDiy3KZl
Guardando archivo Excel en: data/data_medios_20250807_194706.xlsx
Archivo guardado correctamente.
Excel generado con éxito 🎉


In [ ]:
#EJECUCION DE LA FUNCION A TODAS LAS URLS CAPTURADAS PARA EL DIA

for url in lista_url:
    ManagerFaceBookScrapper(url,medio="medios")

___

In [3]:
#LIMPIEZA DATAFRAME Y CONSOLIDADO DIARIO INTERPRETALAB
carpeta = 'data'

archivos = [f for f in os.listdir(carpeta) if f.endswith((".xlsx",".xls"))]
dataframes = []

for archivo in archivos:
    ruta_completa = os.path.join(carpeta,archivo)
    df = pd.read_excel(ruta_completa)
    dataframes.append(df)

df_final = pd.concat(dataframes,ignore_index=True)

mask = df_final["Reacciones"] != 0  # Filtrar filas donde "Número" no sea 0

for idx in df_final[mask].index:  # Iterar sobre las filas filtradas
    partes = df_final.at[idx, "Comentario"].rsplit("\n", 2)  # Dividir de derecha a izquierda
    if len(partes) > 1:
        df_final.at[idx, "Comentario"] = partes[0]  # Asignar el comentario
        df_final.at[idx, "Hora"] = partes[1]        # Asignar la hora
    else:
        df_final.at[idx, "Hora"] = "Pendiente"  # Si no hay salto de línea, dejar "Pendiente"


timestamp = datetime.now().strftime("%d%m%Y")
filename = f'data consolidado/facebook_chile_{timestamp}.xlsx'

# df_final['Codigo'] = df_final['url_facebook'].str.split('/').str[3].astype('float') # crea la columna codigo/que es el ID del Medio de comunicación

# df_final.to_excel(filename,index=False)

In [4]:
df_final.to_excel(filename,index=False)